**Run the following two cells before you begin.**

In [149]:
%autosave 10

Autosaving every 10 seconds


In [150]:
import pandas as pd
import numpy as np

______________________________________________________________________
**First, import your data set and define the sigmoid function.**
<details>
    <summary>Hint:</summary>
    The definition of the sigmoid is $f(x) = \frac{1}{1 + e^{-X}}$.
</details>

In [151]:
# Import the data set
dataset=pd.read_csv(r'https://raw.githubusercontent.com/Sshrut/Data-Science-Intern-2-Mini-Project-Task/master/Task%201%20Data%20set/cleaned_data.csv')
dataset.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month,EDUCATION_CAT,graduate school,high school,others,university
0,798fc410-45c1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,1,university,0,0,0,1
1,8a8c8f3b-8eb4,120000,2,2,2,26,-1,2,0,0,...,1000,1000,0,2000,1,university,0,0,0,1
2,85698822-43f5,90000,2,2,2,34,0,0,0,0,...,1000,1000,1000,5000,0,university,0,0,0,1
3,0737c11b-be42,50000,2,2,1,37,0,0,0,0,...,1200,1100,1069,1000,0,university,0,0,0,1
4,3b7f77cc-dbc0,50000,1,2,1,57,-1,0,-1,0,...,10000,9000,689,679,0,university,0,0,0,1


In [152]:
# Define the sigmoid function
def sigmoid(inp):
    return 1/(1+np.exp(-inp))

**Now, create a train/test split (80/20) with `PAY_1` and `LIMIT_BAL` as features and `default payment next month` as values. Use a random state of 24.**

In [153]:
# Create a train/test split
features=['PAY_1','LIMIT_BAL']
X=dataset[features].values
y=dataset['default payment next month'].values

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=24)

______________________________________________________________________
**Next, import LogisticRegression, with the default options, but set the solver to `'liblinear'`.**

In [154]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(solver='liblinear')

______________________________________________________________________
**Now, train on the training data and obtain predicted classes, as well as class probabilities, using the testing data.**

In [155]:
# Fit the logistic regression model on training data
lr.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [156]:
# Make predictions using `.predict()`
y_pred=lr.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [157]:
# Find class probabilities using `.predict_proba()`
y_prob_pred=lr.predict_proba(X_test)

______________________________________________________________________
**Then, pull out the coefficients and intercept from the trained model and manually calculate predicted probabilities. You'll need to add a column of 1s to your features, to multiply by the intercept.**

In [158]:
# Add column of 1s to features
df_test=pd.DataFrame(X_test,columns=['PAY_1',"LIMIT_BAL"])
df_test['New_Col']=1

In [159]:
# Get coefficients and intercepts from trained model
coef=lr.coef_
print(coef)
intercept=lr.intercept_
intercept

[[ 8.27451187e-11 -6.80876727e-06]]


array([-6.57647457e-11])

In [160]:
# Manually calculate predicted probabilities
df_test['PAY_1']=df_test["PAY_1"]*coef[0][0]
df_test["LIMIT_BAL"]=df_test["LIMIT_BAL"]*coef[0][1]
df_test["New_Col"]=df_test["New_Col"]*intercept

X_final_test=df_test['PAY_1']+df_test['LIMIT_BAL']+df_test['New_Col']
y_pred_manually=sigmoid(X_final_test)
y_pred_manually

0       0.251731
1       0.415703
2       0.203955
3       0.203955
4       0.415703
          ...   
5328    0.278236
5329    0.415703
5330    0.415703
5331    0.172785
5332    0.336066
Length: 5333, dtype: float64

______________________________________________________________________
**Next, using a threshold of `0.5`, manually calculate predicted classes. Compare this to the class predictions output by scikit-learn.**

In [161]:
# Manually calculate predicted classes
y_manually=y_pred_manually>=0.5
y_manually.replace(to_replace=[True,False],value=[1,0],inplace=True)
y_manually

0       0
1       0
2       0
3       0
4       0
       ..
5328    0
5329    0
5330    0
5331    0
5332    0
Length: 5333, dtype: object

In [162]:
# Compare to scikit-learn's predicted classes
accuracy_of_same_no_of_prediction=sum(sum(y_manually.values.reshape(len(y_manually),1)==y_pred.reshape(len(y_pred),1)))/len(y_test)*100

print(f"Same no of prediction : {accuracy_of_same_no_of_prediction} % ")

Same no of prediction : 100.0 % 


______________________________________________________________________
**Finally, calculate ROC AUC using both scikit-learn's predicted probabilities, and your manually predicted probabilities, and compare.**

In [163]:
# Use scikit-learn's predicted probabilities to calculate ROC AUC
from sklearn.metrics import roc_auc_score
score= roc_auc_score(y_test,y_prob_pred[:,1])
score

0.627207450280691

In [164]:
# Use manually calculated predicted probabilities to calculate ROC AUC
score= roc_auc_score(y_test,y_pred_manually)
score

0.627207450280691

In [165]:
print("ROC AUC using both scikit-learn's predicted probabilities, and manually predicted probabilities are 'EXCATLY SAME '")

ROC AUC using both scikit-learn's predicted probabilities, and manually predicted probabilities are 'EXCATLY SAME '
